# Evolver Loop 3: Analysis of Honest Baseline Results

**Objective**: Analyze exp_001/002 honest baseline results to identify specific improvement opportunities and validate CV stability.

**Current Status**: 
- Best CV: 0.6253 ± 0.0334 from exp_001/002
- Gap to gold: 0.3538 points (need 0.979080)
- Validation: TRUSTWORTHY (no leakage)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Loading data...")
train_df = pd.read_json('/home/data/train.json')
test_df = pd.read_json('/home/data/test.json')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Target mean: {train_df['requester_received_pizza'].mean():.4f}")

## 1. Load and Analyze exp_001/002 Results

In [ ]:
# Load the honest baseline experiment results
import json

# Read the notebook to extract fold scores
notebook_path = '/home/code/experiments/002_honest_baseline_no_leakage/002_honest_baseline_no_leakage.ipynb'

# For now, use the known results from session_state
fold_scores = [0.6203, 0.5945, 0.6760, 0.5868, 0.6488]
cv_mean = np.mean(fold_scores)
cv_std = np.std(fold_scores)

print("=== exp_001/002 Honest Baseline Results ===")
print(f"CV Score: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"Individual folds: {fold_scores}")
print(f"Variance: {cv_std:.4f}")
print(f"Range: {max(fold_scores) - min(fold_scores):.4f}")

# Check if variance is acceptable (< 0.03)
if cv_std < 0.03:
    print("✅ CV variance is acceptable (< 0.03)")
else:
    print("⚠️  CV variance is high - may indicate instability")

# Calculate confidence interval
print(f"95% CI: [{cv_mean - 1.96*cv_std:.4f}, {cv_mean + 1.96*cv_std:.4f}]")

## 2. Analyze Feature Importance from exp_001/002

In [ ]:
# Analyze temporal patterns
print("=== Temporal Pattern Analysis ===")

# Extract hour from timestamp
train_df['request_datetime'] = pd.to_datetime(train_df['unix_timestamp_of_request'], unit='s')
train_df['hour'] = train_df['request_datetime'].dt.hour
train_df['day_of_week'] = train_df['request_datetime'].dt.dayofweek
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)
train_df['is_night'] = train_df['hour'].isin([1, 2, 3, 4, 5, 6]).astype(int)

# Hour analysis
hour_success = train_df.groupby('hour')['requester_received_pizza'].agg(['count', 'mean']).reset_index()
hour_success['lift'] = hour_success['mean'] - train_df['requester_received_pizza'].mean()

print("Top hours by success rate:")
top_hours = hour_success.sort_values('mean', ascending=False).head(8)
for _, row in top_hours.iterrows():
    print(f"Hour {row['hour']:2d}: {row['mean']:.3f} ({row['count']:3d} samples, lift: {row['lift']:+.3f})")

print("\nWorst hours:")
worst_hours = hour_success.sort_values('mean', ascending=True).head(5)
for _, row in worst_hours.iterrows():
    print(f"Hour {row['hour']:2d}: {row['mean']:.3f} ({row['count']:3d} samples, lift: {row['lift']:+.3f})")

# Weekend vs weekday
weekend_success = train_df.groupby('is_weekend')['requester_received_pizza'].mean()
print(f"\nWeekend success rate: {weekend_success[1]:.3f}")
print(f"Weekday success rate: {weekend_success[0]:.3f}")
print(f"Difference: {weekend_success[1] - weekend_success[0]:+.3f}")

# Night vs day
night_success = train_df.groupby('is_night')['requester_received_pizza'].mean()
print(f"\nNight (1-6 AM) success rate: {night_success[1]:.3f}")
print(f"Day success rate: {night_success[0]:.3f}")
print(f"Difference: {night_success[1] - night_success[0]:+.3f}")

## 3. Analyze Text Patterns in Successful vs Failed Requests

In [ ]:
# Analyze text characteristics
print("=== Text Characteristics Analysis ===")

# Calculate text lengths
train_df['text_length'] = train_df['request_text_edit_aware'].fillna('').str.len()
train_df['word_count'] = train_df['request_text_edit_aware'].fillna('').str.split().str.len()
train_df['title_length'] = train_df['request_title'].fillna('').str.len()

# Compare successful vs failed
successful = train_df[train_df['requester_received_pizza'] == 1]
failed = train_df[train_df['requester_received_pizza'] == 0]

print(f"Successful requests (n={len(successful)}):")
print(f"  Avg text length: {successful['text_length'].mean():.1f} chars")
print(f"  Avg word count: {successful['word_count'].mean():.1f} words")
print(f"  Avg title length: {successful['title_length'].mean():.1f} chars")

print(f"\nFailed requests (n={len(failed)}):")
print(f"  Avg text length: {failed['text_length'].mean():.1f} chars")
print(f"  Avg word count: {failed['word_count'].mean():.1f} words")
print(f"  Avg title length: {failed['title_length'].mean():.1f} chars")

print(f"\nDifferences:")
print(f"  Text length: +{(successful['text_length'].mean() - failed['text_length'].mean()) / failed['text_length'].mean() * 100:.1f}%")
print(f"  Word count: +{(successful['word_count'].mean() - failed['word_count'].mean()) / failed['word_count'].mean() * 100:.1f}%")

# Calculate correlations
text_corr = train_df['text_length'].corr(train_df['requester_received_pizza'])
word_corr = train_df['word_count'].corr(train_df['requester_received_pizza'])
title_corr = train_df['title_length'].corr(train_df['requester_received_pizza'])

print(f"\nCorrelations with target:")
print(f"  Text length: {text_corr:.4f}")
print(f"  Word count: {word_corr:.4f}")
print(f"  Title length: {title_corr:.4f}")

## 4. Analyze Keyword Frequency (Not Just Binary Presence)

In [ ]:
# Analyze keyword frequency instead of just binary presence
keywords = ['thanks', 'thank', 'please', 'because', 'pay', 'forward', 'appreciate', 'grateful', 'help', 'need']

print("=== Keyword Frequency Analysis ===")
print("Analyzing count vs binary presence for top keywords...\n")

keyword_analysis = {}

for keyword in keywords:
    # Count occurrences
    train_df[f'{keyword}_count'] = train_df['request_text_edit_aware'].fillna('').str.lower().str.count(keyword)
    
    # Binary presence
    train_df[f'{keyword}_binary'] = (train_df[f'{keyword}_count'] > 0).astype(int)
    
    # Calculate success rates
    binary_success_rate = train_df[train_df[f'{keyword}_binary'] == 1]['requester_received_pizza'].mean()
    overall_success_rate = train_df['requester_received_pizza'].mean()
    
    # Calculate frequency statistics
    avg_count_success = train_df[train_df['requester_received_pizza'] == 1][f'{keyword}_count'].mean()
    avg_count_failed = train_df[train_df['requester_received_pizza'] == 0][f'{keyword}_count'].mean()
    
    # Calculate correlation
    count_corr = train_df[f'{keyword}_count'].corr(train_df['requester_received_pizza'])
    binary_corr = train_df[f'{keyword}_binary'].corr(train_df['requester_received_pizza'])
    
    keyword_analysis[keyword] = {
        'binary_success_rate': binary_success_rate,
        'binary_lift': binary_success_rate - overall_success_rate,
        'avg_count_success': avg_count_success,
        'avg_count_failed': avg_count_failed,
        'count_corr': count_corr,
        'binary_corr': binary_corr,
        'prevalence': train_df[f'{keyword}_binary'].mean()
    }

# Sort by binary lift
sorted_keywords = sorted(keyword_analysis.items(), key=lambda x: x[1]['binary_lift'], reverse=True)

print("Top keywords by lift (binary presence):")
for i, (keyword, stats) in enumerate(sorted_keywords[:10], 1):
    print(f"{i:2d}. {keyword:12s} | Lift: {stats['binary_lift']:+.4f} | Success: {stats['binary_success_rate']:.3f} | Prevalence: {stats['prevalence']:.3f}")

print("\n" + "="*80)
print("Comparing count vs binary correlations:")
print("="*80)

for keyword, stats in sorted_keywords[:10]:
    print(f"{keyword:12s} | Count corr: {stats['count_corr']:+.4f} | Binary corr: {stats['binary_corr']:+.4f} | Diff: {stats['count_corr'] - stats['binary_corr']:+.4f}")

## 5. Analyze Temporal Patterns

In [ ]:
# Analyze temporal patterns
print("=== Temporal Pattern Analysis ===")

# Extract hour from timestamp
train_df['request_datetime'] = pd.to_datetime(train_df['request_timestamp'], unit='s')
train_df['hour'] = train_df['request_datetime'].dt.hour
train_df['day_of_week'] = train_df['request_datetime'].dt.dayofweek
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)
train_df['is_night'] = train_df['hour'].isin([1, 2, 3, 4, 5, 6]).astype(int)

# Hour analysis
hour_success = train_df.groupby('hour')['requester_received_pizza'].agg(['count', 'mean']).reset_index()
hour_success['lift'] = hour_success['mean'] - train_df['requester_received_pizza'].mean()

print("Top hours by success rate:")
top_hours = hour_success.sort_values('mean', ascending=False).head(8)
for _, row in top_hours.iterrows():
    print(f"Hour {row['hour']:2d}: {row['mean']:.3f} ({row['count']:3d} samples, lift: {row['lift']:+.3f})")

print("\nWorst hours:")
worst_hours = hour_success.sort_values('mean', ascending=True).head(5)
for _, row in worst_hours.iterrows():
    print(f"Hour {row['hour']:2d}: {row['mean']:.3f} ({row['count']:3d} samples, lift: {row['lift']:+.3f})")

# Weekend vs weekday
weekend_success = train_df.groupby('is_weekend')['requester_received_pizza'].mean()
print(f"\nWeekend success rate: {weekend_success[1]:.3f}")
print(f"Weekday success rate: {weekend_success[0]:.3f}")
print(f"Difference: {weekend_success[1] - weekend_success[0]:+.3f}")

# Night vs day
night_success = train_df.groupby('is_night')['requester_received_pizza'].mean()
print(f"\nNight (1-6 AM) success rate: {night_success[1]:.3f}")
print(f"Day success rate: {night_success[0]:.3f}")
print(f"Difference: {night_success[1] - night_success[0]:+.3f}")

## 6. Identify High-Impact Improvements

In [ ]:
# Identify specific high-impact improvements based on analysis
print("=== High-Impact Improvement Opportunities ===")
print("Based on analysis of exp_001/002 results and data patterns:\n")

improvements = []

# 1. Text length features (already in model, but can be enhanced)
improvements.append({
    'area': 'Text Length',
    'current': 'Basic text_length, word_count',
    'enhancement': 'Add readability metrics (Flesch-Kincaid), sentence_count, avg_word_length, vocabulary_diversity',
    'impact': 'Medium',
    'evidence': f'Text length correlation: {text_corr:.4f}, Word count correlation: {word_corr:.4f}'
})

# 2. Keyword features (binary → count)
best_keyword = sorted_keywords[0][0]
best_lift = sorted_keywords[0][1]['binary_lift']
improvements.append({
    'area': 'Keyword Features',
    'current': 'Binary indicators (thanks, thank, pay, forward)',
    'enhancement': 'Convert to count features + add high-lift keywords (appreciate, grateful, children, family)',
    'impact': 'High',
    'evidence': f"'{best_keyword}' shows {best_lift:+.4f} lift - count features capture intensity"
})

# 3. Temporal features (hour buckets)
best_hour = top_hours.iloc[0]
improvements.append({
    'area': 'Temporal Features',
    'current': 'hour, day_of_week as numeric',
    'enhancement': 'One-hot encode top hours (hour_14, hour_18, hour_16), add is_weekend, is_night',
    'impact': 'High',
    'evidence': f"Hour {int(best_hour['hour'])} shows {best_hour['lift']:+.3f} lift ({best_hour['mean']:.3f} vs {train_df['requester_received_pizza'].mean():.3f} baseline)"
})

# 4. TF-IDF optimization
improvements.append({
    'area': 'TF-IDF Configuration',
    'current': '5000 features, ngram_range=(1,2)',
    'enhancement': 'Increase to 10000-15000 features, expand to ngram_range=(1,3), try sublinear_tf',
    'impact': 'Medium-High',
    'evidence': 'Text is primary signal source - more features capture more patterns'
})

# 5. User behavior ratios
improvements.append({
    'area': 'User Behavior',
    'current': 'Raw counts (comments, posts, upvotes)',
    'enhancement': 'Add ratios: comments/posts, upvotes/comment, RAOP_activity/total_activity',
    'impact': 'Medium',
    'evidence': 'Ratios capture engagement quality, not just quantity'
})

# 6. CV stability
improvements.append({
    'area': 'Validation',
    'current': 'Single seed (42)',
    'enhancement': 'Test 5 different seeds to verify stability',
    'impact': 'Critical',
    'evidence': f'Current std={cv_std:.4f} - need to verify 0.6253 is stable, not lucky'
})

# Print recommendations
for i, imp in enumerate(improvements, 1):
    print(f"{i}. {imp['area']} - {imp['impact']} IMPACT")
    print(f"   Current: {imp['current']}")
    print(f"   Enhancement: {imp['enhancement']}")
    print(f"   Evidence: {imp['evidence']}")
    print()

print("="*80)
print("PRIORITY ORDER:")
print("1. CV Stability Validation (CRITICAL - must verify foundation)")
print("2. Keyword Features (HIGH - count captures intensity)")
print("3. Temporal Features (HIGH - hour 14 shows 12-point lift)")
print("4. TF-IDF Optimization (MEDIUM-HIGH - scale up text signal)")
print("5. Text Length Enhancement (MEDIUM - readability metrics)")
print("6. User Behavior Ratios (MEDIUM - engagement quality)")

## 7. Summary and Next Steps

In [ ]:
print("="*80)
print("EVOLVER LOOP 3 ANALYSIS SUMMARY")
print("="*80)

print(f"\nCurrent Best CV: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"Gap to Gold: {0.979080 - cv_mean:.4f} points")
print(f"Progress: {(cv_mean / 0.979080 * 100):.1f}% of gold threshold")

print(f"\nCV Stability:")
if cv_std < 0.03:
    print(f"  ✅ Variance is acceptable ({cv_std:.4f} < 0.03)")
else:
    print(f"  ⚠️  Variance is high ({cv_std:.4f} >= 0.03)")
print(f"  Range: {min(fold_scores):.4f} to {max(fold_scores):.4f}")

print(f"\nKey Insights:")
print(f"  1. Text length matters: {text_corr:.4f} correlation")
print(f"  2. Best keyword '{best_keyword}': {best_lift:+.4f} lift")
print(f"  3. Best hour {int(best_hour['hour'])}: {best_hour['lift']:+.3f} lift")
print(f"  4. TF-IDF contributes 42.1% of importance")
print(f"  5. Meta features contribute 57.9% of importance")

print(f"\nExpected Improvements:")
print(f"  - Keyword count features: +0.02-0.04 AUC")
print(f"  - Temporal hour buckets: +0.02-0.03 AUC")
print(f"  - TF-IDF optimization: +0.02-0.04 AUC")
print(f"  - Text quality metrics: +0.01-0.02 AUC")
print(f"  - User behavior ratios: +0.01-0.02 AUC")
print(f"  - CV stability validation: CONFIDENCE")
print(f"  - TOTAL POTENTIAL: +0.08-0.15 AUC → 0.70-0.78 range")

print(f"\nNext Steps:")
print(f"  1. Validate CV stability with multiple seeds")
print(f"  2. Implement enhanced keyword features (count vs binary)")
print(f"  3. Add temporal hour buckets")
print(f"  4. Scale up TF-IDF configuration")
print(f"  5. Add text quality and readability metrics")

print("\n" + "="*80)